# Demo IoT Sensor Data Input
**Simulation the sensor sensing environment and sending data into our system, which setup in [1.Setup_Env](./1.Setup_Env.ipynb), Step 1~5.**

## 1. Import Required Lib

In [49]:
import json
import csv
import requests
import time
import os
import copy

## 2. Setup Env Var
**\_\_AIOTCORE\_\_ is the aiot-core publice interface's IP address of aiot-core container.**

**\_\_IOTA\_UL\_\_  and \_\_IOTA\_JSON\_\_ are the iot-agent publice interface's IP address of both container.**

In [50]:
__AIOTCORE__="http://127.0.0.1:9250"
__IOTA_UL__="http://127.0.0.1:7896"
__IOTA_JSON__="http://127.0.0.1:7897"

## 3. Show All Usuable Dataset Name

In [51]:
dirlist=os.listdir("../../Dataset/NAB/")
try:
    dirlist.remove("README.md")
except:
    pass
try:
    dirlist.remove("combined_windows.json")
except:
    pass
filelist=[]
for dirname in dirlist:
    try:
        file=os.listdir(f"../../Dataset/NAB/{dirname}/")
        for i in file:
            filelist.append(f"{dirname}/{i}")
    except:
        pass
filedict={x:[] for x in filelist}
print("available dataset name:")
for file_name in filedict:
    print(file_name)

available dataset name:
artificialWithAnomaly/art_daily_jumpsdown.csv
artificialWithAnomaly/art_daily_flatmiddle.csv
artificialWithAnomaly/art_daily_nojump.csv
artificialWithAnomaly/art_load_balancer_spikes.csv
artificialWithAnomaly/art_daily_jumpsup.csv
artificialWithAnomaly/art_increase_spike_density.csv
realTraffic/occupancy_t4013.csv
realTraffic/speed_7578.csv
realTraffic/speed_6005.csv
realTraffic/TravelTime_387.csv
realTraffic/speed_t4013.csv
realTraffic/TravelTime_451.csv
realTraffic/occupancy_6005.csv
realKnownCause/ambient_temperature_system_failure.csv
realKnownCause/machine_temperature_system_failure.csv
realKnownCause/rogue_agent_key_updown.csv
realKnownCause/ec2_request_latency_system_failure.csv
realKnownCause/rogue_agent_key_hold.csv
realKnownCause/cpu_utilization_asg_misconfiguration.csv
realKnownCause/nyc_taxi.csv
realAdExchange/exchange-3_cpc_results.csv
realAdExchange/exchange-2_cpc_results.csv
realAdExchange/exchange-3_cpm_results.csv
realAdExchange/exchange-2_cpm_r

## 4. Pick Wanted Dataset and Load Data Into Memory

In [52]:
wanted=["artificialWithAnomaly/art_daily_jumpsdown.csv",
        "artificialWithAnomaly/art_daily_jumpsup.csv",
        "realKnownCause/ec2_request_latency_system_failure.csv",
        "realKnownCause/machine_temperature_system_failure.csv"]

for dirname in dirlist:
    filelist=os.listdir(f"../../Dataset/NAB/{dirname}/")
    for file in filelist:
        if f"{dirname}/{file}" not in wanted:
            continue
        with open(f"../../Dataset/NAB/{dirname}/{file}","r") as f:
            data=f.readlines()
            data.pop(0)
            for idx,line in enumerate(data):
                time=line.split(",")[0].replace(" ","T")[0:-3]
                value=float(line.split(",")[1])
                filedict[f"{dirname}/{file}"].append((time,value))
print("Data Loaded:")
for file_name,datalist in filedict.items():
    if file_name not in wanted:
        continue
    print(f"{file_name}:")
    for d in datalist[0:10]:
        print(d)
    print("...")
    

Data Loaded:
artificialWithAnomaly/art_daily_jumpsdown.csv:
('2014-04-01T00:00', 18.090486228499998)
('2014-04-01T00:05', 20.359842585899997)
('2014-04-01T00:10', 21.105469847200002)
('2014-04-01T00:15', 21.1515852522)
('2014-04-01T00:20', 18.1371405424)
('2014-04-01T00:25', 21.6256605306)
('2014-04-01T00:30', 21.6738066877)
('2014-04-01T00:35', 20.2455614926)
('2014-04-01T00:40', 19.9283327955)
('2014-04-01T00:45', 18.1794698523)
...
artificialWithAnomaly/art_daily_jumpsup.csv:
('2014-04-01T00:00', 19.761251902999998)
('2014-04-01T00:05', 20.500833287)
('2014-04-01T00:10', 19.9616414445)
('2014-04-01T00:15', 21.4902660734)
('2014-04-01T00:20', 20.1877394098)
('2014-04-01T00:25', 19.9231256718)
('2014-04-01T00:30', 21.698403961700002)
('2014-04-01T00:35', 20.8787583842)
('2014-04-01T00:40', 18.4461996294)
('2014-04-01T00:45', 18.7108178448)
...
realKnownCause/machine_temperature_system_failure.csv:
('2013-12-02T21:15', 73.96732207)
('2013-12-02T21:20', 74.93588199999998)
('2013-12-02T2

## Get Iot Agent Setting From AIOT-CORE

In [53]:
import pprint
deviceInfo={}
service_group=requests.get(f"{__AIOTCORE__}/service-groups").json()['value']
for SG in service_group:
    devices=requests.get(f"{__AIOTCORE__}/service-groups/{SG}").json()["Device"]
    for DeviceName in devices:
        url=f"{__AIOTCORE__}/devices/{SG}/{DeviceName}"
        data=requests.get(url).json()
        agentURL=data["iotagent_config"]["Iot-Agent-Url"].split(":")[0:2]+[str(data["iotagent_config"]["Device-Port"])]
        agentURL=":".join(agentURL)
        if "7897" in agentURL:
            agentURL=__IOTA_JSON__
        else:
            agentURL=__IOTA_UL__
        apikey=data["iotagent_config"]["apikey"]
        Resource=data["iotagent_config"]["Resource"]
        DeviceID=f"{SG}:{DeviceName}"
        filename=data["static_attributes"]["fieldName"]+".csv"
        for i in filedict:
            if filename in i:
                filename=i
                break
        deviceInfo[filename]={"agentURL":agentURL,"apikey":apikey,"Resource":Resource,"DeviceID":DeviceID}
pprint.pprint(deviceInfo)
filedata=copy.deepcopy(filedict)

{'artificialWithAnomaly/art_daily_jumpsdown.csv': {'DeviceID': 'SG_Artifical:Sensor01',
                                                   'Resource': '/iot/d',
                                                   'agentURL': 'http://140.116.72.250:7896',
                                                   'apikey': 'vpCNt19G3gNJPfE21V7w6oUcYY'},
 'artificialWithAnomaly/art_daily_jumpsup.csv': {'DeviceID': 'SG_Artifical:Sensor02',
                                                 'Resource': '/iot/json',
                                                 'agentURL': 'http://140.116.72.250:7897',
                                                 'apikey': 'wiUIlRLOAuS0Nv3mCxrhJnAsVP'},
 'realKnownCause/ec2_request_latency_system_failure.csv': {'DeviceID': 'SG_Real:Sensor01',
                                                           'Resource': '/iot/d',
                                                           'agentURL': 'http://140.116.72.250:7896',
                                        

## Start Feeding Data
**Delay Period is the IoT Device sensing cycle time in second, must not be smaller than 4**

In [ ]:
import time 
delay_period=4


count=0
while count!=len(wanted)-1:
    count=0
    for file,content in filedata.items():
        try:
            data=content.pop(0)
        except:
            count+=1
            continue
        device=deviceInfo[file]
        agentURL=device["agentURL"]
        apikey=device["apikey"]
        Resource=device["Resource"]
        DeviceID=device["DeviceID"]
        if "7897" in agentURL:
            header = {"Content-Type": "application/json"}
            body = {"COUNT" : str(data[1]) , "TS": data[0]}
            print(f"Request URL:{agentURL}{Resource}?k={apikey}&i={DeviceID}")
            print(f"Request Data:{body}")
            r = requests.post(agentURL+Resource+"?k="+apikey+"&i="+DeviceID,
                headers=header,
                data=json.dumps(body),
            )
            print(r.status_code,r.text)
        else:
            header = {"Content-Type": "text/plain"}
            body = "COUNT|" + str(data[1]) + "|TS|" + data[0]
            print(f"Request URL:{agentURL}{Resource}?k={apikey}&i={DeviceID}")
            print(f"Request Data:{body}")
            r = requests.post(agentURL+Resource+"?k="+apikey+"&i="+DeviceID,
                headers=header,
                data=body
            )
            print(r.status_code,r.text)
    time.sleep(int(delay_period))

print("End")

Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Artifical:Sensor01
Request Data:COUNT|18.090486228499998|TS|2014-04-01T00:00
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Artifical:Sensor02
Request Data:{'COUNT': '19.761251902999998', 'TS': '2014-04-01T00:00'}
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Real:Sensor02
Request Data:{'COUNT': '73.96732207', 'TS': '2013-12-02T21:15'}
200 
Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Real:Sensor01
Request Data:COUNT|45.868|TS|2014-03-07T03:41
200 
Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Artifical:Sensor01
Request Data:COUNT|20.359842585899997|TS|2014-04-01T00:05
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Artifical:Sensor02
Request Data:{'COUNT': '20.500833287', 'TS': '2014-04-01T00:05'}
200 
Request URL:http://14

Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Artifical:Sensor01
Request Data:COUNT|19.6155342675|TS|2014-04-01T01:05
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Artifical:Sensor02
Request Data:{'COUNT': '20.217820911500002', 'TS': '2014-04-01T01:05'}
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Real:Sensor02
Request Data:{'COUNT': '80.80262407', 'TS': '2013-12-02T22:20'}
200 
Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Real:Sensor01
Request Data:COUNT|46.31399999999999|TS|2014-03-07T04:46
200 
Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Artifical:Sensor01
Request Data:COUNT|20.7343233333|TS|2014-04-01T01:10
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Artifical:Sensor02
Request Data:{'COUNT': '20.527731851400002', 'TS': '2014-04-01T01:10'}
200 
Request URL:ht

Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Artifical:Sensor01
Request Data:COUNT|19.2570803929|TS|2014-04-01T02:10
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Artifical:Sensor02
Request Data:{'COUNT': '19.530564620299998', 'TS': '2014-04-01T02:10'}
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Real:Sensor02
Request Data:{'COUNT': '81.2580527', 'TS': '2013-12-02T23:25'}
200 
Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Real:Sensor01
Request Data:COUNT|42.501999999999995|TS|2014-03-07T05:51
200 
Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Artifical:Sensor01
Request Data:COUNT|20.848435731400002|TS|2014-04-01T02:15
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Artifical:Sensor02
Request Data:{'COUNT': '20.4531455753', 'TS': '2014-04-01T02:15'}
200 
Request URL:ht

Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Artifical:Sensor01
Request Data:COUNT|19.3445219219|TS|2014-04-01T03:15
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Artifical:Sensor02
Request Data:{'COUNT': '18.6391422485', 'TS': '2014-04-01T03:15'}
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Real:Sensor02
Request Data:{'COUNT': '83.57965349', 'TS': '2013-12-03T00:30'}
200 
Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Real:Sensor01
Request Data:COUNT|46.478|TS|2014-03-07T06:56
200 
Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Artifical:Sensor01
Request Data:COUNT|18.89490293|TS|2014-04-01T03:20
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Artifical:Sensor02
Request Data:{'COUNT': '19.5321534017', 'TS': '2014-04-01T03:20'}
200 
Request URL:http://140.116.72.250:789

Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Artifical:Sensor01
Request Data:COUNT|18.7145268525|TS|2014-04-01T04:20
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Artifical:Sensor02
Request Data:{'COUNT': '20.0388908223', 'TS': '2014-04-01T04:20'}
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Real:Sensor02
Request Data:{'COUNT': '84.34921079', 'TS': '2013-12-03T01:35'}
200 
Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Real:Sensor01
Request Data:COUNT|43.618|TS|2014-03-07T08:01
200 
Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Artifical:Sensor01
Request Data:COUNT|19.6175048029|TS|2014-04-01T04:25
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Artifical:Sensor02
Request Data:{'COUNT': '21.6116758929', 'TS': '2014-04-01T04:25'}
200 
Request URL:http://140.116.72.250:7

...

Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Artifical:Sensor01
Request Data:COUNT|71.1868242635|TS|2014-04-01T09:45
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Artifical:Sensor02
Request Data:{'COUNT': '68.9065226554', 'TS': '2014-04-01T09:45'}
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Real:Sensor02
Request Data:{'COUNT': '83.21031771', 'TS': '2013-12-03T07:00'}
200 
Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Real:Sensor01
Request Data:COUNT|45.773999999999994|TS|2014-03-07T13:26
200 
Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Artifical:Sensor01
Request Data:COUNT|68.5181183224|TS|2014-04-01T09:50
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Artifical:Sensor02
Request Data:{'COUNT': '64.8937453607', 'TS': '2014-04-01T09:50'}
200 
Request URL:http://140.

200 
Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Artifical:Sensor01
Request Data:COUNT|74.2964790823|TS|2014-04-01T10:50
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Artifical:Sensor02
Request Data:{'COUNT': '74.3401531544', 'TS': '2014-04-01T10:50'}
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Real:Sensor02
Request Data:{'COUNT': '84.15133223', 'TS': '2013-12-03T08:05'}
200 
Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Real:Sensor01
Request Data:COUNT|42.88800000000001|TS|2014-03-07T14:31
200 
Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Artifical:Sensor01
Request Data:COUNT|83.0001929198|TS|2014-04-01T10:55
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Artifical:Sensor02
Request Data:{'COUNT': '82.9596010938', 'TS': '2014-04-01T10:55'}
200 
Request URL:http://

Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Artifical:Sensor01
Request Data:COUNT|79.6632095013|TS|2014-04-01T11:55
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Artifical:Sensor02
Request Data:{'COUNT': '85.426067155', 'TS': '2014-04-01T11:55'}
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Real:Sensor02
Request Data:{'COUNT': '85.86375607', 'TS': '2013-12-03T09:10'}
200 
Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Real:Sensor01
Request Data:COUNT|44.882|TS|2014-03-07T15:36
200 
Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Artifical:Sensor01
Request Data:COUNT|76.9663781954|TS|2014-04-01T12:00
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Artifical:Sensor02
Request Data:{'COUNT': '80.08686564029999', 'TS': '2014-04-01T12:00'}
200 
Request URL:http://140.116.72.25

Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Artifical:Sensor01
Request Data:COUNT|82.0077546958|TS|2014-04-01T13:00
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Artifical:Sensor02
Request Data:{'COUNT': '86.3926524705', 'TS': '2014-04-01T13:00'}
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Real:Sensor02
Request Data:{'COUNT': '81.50854903', 'TS': '2013-12-03T10:15'}
200 
Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Real:Sensor01
Request Data:COUNT|45.693999999999996|TS|2014-03-07T16:41
200 
Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Artifical:Sensor01
Request Data:COUNT|86.08517365450001|TS|2014-04-01T13:05
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Artifical:Sensor02
Request Data:{'COUNT': '83.47372499939999', 'TS': '2014-04-01T13:05'}
200 
Request URL:htt

Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Artifical:Sensor01
Request Data:COUNT|78.9661264483|TS|2014-04-01T14:05
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Artifical:Sensor02
Request Data:{'COUNT': '86.883522504', 'TS': '2014-04-01T14:05'}
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Real:Sensor02
Request Data:{'COUNT': '76.18233367', 'TS': '2013-12-03T11:20'}
200 
Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Real:Sensor01
Request Data:COUNT|45.63399999999999|TS|2014-03-07T17:46
200 
Request URL:http://140.116.72.250:7896/iot/d?k=vpCNt19G3gNJPfE21V7w6oUcYY&i=SG_Artifical:Sensor01
Request Data:COUNT|80.25720329090001|TS|2014-04-01T14:10
200 
Request URL:http://140.116.72.250:7897/iot/json?k=wiUIlRLOAuS0Nv3mCxrhJnAsVP&i=SG_Artifical:Sensor02
Request Data:{'COUNT': '82.0168805171', 'TS': '2014-04-01T14:10'}
200 
Request URL:http://14